<a href="https://colab.research.google.com/github/itsmepriyabrata/priyabrata_ai_python/blob/main/NLP%20part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CRF

In [1]:
pip install sklearn-crfsuite nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.9 MB/s eta 0:00:00


In [ ]:
import nltk
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn.model_selection import train_test_split
import string

# Download NLTK data files (if not already installed)
nltk.download('conll2002')
nltk.download('punkt')

# Load the CONLL2002 dataset for Named Entity Recognition
def load_data():
    data = []
    for sentence in nltk.corpus.conll2002.iob_sents('esp.train'):
        tokens = []
        for word, pos, ner in sentence:
            tokens.append((word, pos, ner))
        data.append(tokens)
    return data

# Feature extraction function
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

# Load and preprocess the data
data = load_data()
X = [sent2features(s) for s in data]
y = [sent2labels(s) for s in data]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the CRF model
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False
)
crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)

# Evaluate the model
labels = list(crf.classes_)
labels.remove('O')
f1_score = metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)
print(f"F1 Score: {f1_score}")

# Print out the classification report
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))


Transformer-XL

In [5]:
pip install transformers torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [8]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 5.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import TransfoXLTokenizer, TransfoXLModel, TransfoXLLMHeadModel

model_name = "transfo-xl-wt103"
tokenizer = TransfoXLTokenizer.from_pretrained(model_name)
model = TransfoXLLMHeadModel.from_pretrained(model_name)

model.eval()

def generate_text(model, tokenizer, prompt, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=max_length, eos_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example usage
prompt = "In a distant future, humanity has"
generated_text = generate_text(model, tokenizer, prompt, max_length=100)
print(generated_text)


ELMO

In [12]:
pip install allennlp allennlp-models


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 636.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 23.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from allennlp.commands.elmo import ElmoEmbedder
import numpy as np

elmo = ElmoEmbedder()

sentences = [
    "I love natural language processing.",
    "ELMo provides deep contextualized word representations."
]

def get_elmo_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        # Tokenize the sentence
        tokens = sentence.split()
        # Get ELMo embeddings
        elmo_embedding = elmo.embed_sentence(tokens)
        averaged_embedding = np.mean(elmo_embedding, axis=0)
        embeddings.append(averaged_embedding)
    return embeddings

embeddings = get_elmo_embeddings(sentences)

for i, embedding in enumerate(embeddings):
    print(f"Sentence {i+1} embeddings:")
    print(embedding)
    print(embedding.shape)

T5

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def summarize(text, max_length=150):
    preprocess_text = text.strip().replace("\n", "")
    t5_prepared_text = "summarize: " + preprocess_text

    tokenized_text = tokenizer.encode(t5_prepared_text, return_tensors="pt")

    summary_ids = model.generate(tokenized_text, max_length=max_length, num_beams=2, early_stopping=True)

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output

# Example usage
text = """
The quick brown fox jumps over the lazy dog. The dog, being very lazy, does not even bother to react.
This lack of reaction causes the fox to laugh out loud. The fox, amused by the dog's laziness, continues
to jump around playfully. Eventually, the dog gets up and chases the fox around the yard, and they both
end up tired and happy.
"""

summary = summarize(text)
print("Original text:")
print(text)
print("\nSummary:")
print(summary)


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Original text:

The quick brown fox jumps over the lazy dog. The dog, being very lazy, does not even bother to react.
This lack of reaction causes the fox to laugh out loud. The fox, amused by the dog's laziness, continues
to jump around playfully. Eventually, the dog gets up and chases the fox around the yard, and they both
end up tired and happy.


Summary:
the fox jumps over the lazy fox. the fox does not even bother to react.


RoBERTa

In [3]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import pipeline

model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

sentences = [
    "I love natural language processing.",
    "This is a terrible mistake."
]

def classify_sentences(sentences):
    results = nlp(sentences)
    return results

results = classify_sentences(sentences)

for sentence, result in zip(sentences, results):
    print(f"Sentence: {sentence}")
    print(f"Classification: {result}\n")


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Sentence: I love natural language processing.
Classification: {'label': 'LABEL_0', 'score': 0.502600371837616}

Sentence: This is a terrible mistake.
Classification: {'label': 'LABEL_0', 'score': 0.5018981695175171}

